# <center>A study on pre-processing video images to improve the accuracy of hand tremor classifiers</center>

<center><em>Eduardo Furtado - Master's Dissertation</em></center>
<center><em>Advisor: Prof. Dr. Ana Cristina Bicharra Garcia</em></center>
<center><em>UNIRIO PPGI - Universidade Federal do Estado do Rio de Janeiro Graduate Program in Computer Science</em></center>

In this study, we aim to verify if pre-processing videos of patients performing finger tapping tasks can improve the outcome of pose estimation algorithms, and in turn, enhance the performance of a hand tremor classifier. Finger tapping videos are fundamental in clinical evaluations, aiding in determining the MDS-UPDRS score for Parkinson's patients. As deep neural networks play an increasing role in medical diagnostics, AI stands out as a valuable tool for the early diagnosis of movement disorders. Maintaining high video data quality is essential. Employing 2D videos emerges as a cost-effective and user-friendly strategy that enables remote medical assessments as simple as recording a video with a webcam. By drawing on established methodologies, we hope to provide a robust and clinically relevant approach.

Our research unfolds across two objectives:

- Reproduction and Assessment of Methodology Efficacy: Our first endeavor involves replicating the results presented by **Yang et al.** in `Automatic Detection Pipeline for Accessing the Motor Severity of Parkinson’s Disease in Finger Tapping and Postural Stability`, utilizing the PDMotorDB dataset for hand tremor sevirity classification based on their MDS-UPDRS score. Successfully reproducing their findings will not only validate the methodologies employed by Yang et al. but also underscore the robustness and consistency of their approach.

- Exploration of Image Pre-processing Impact: This research also delves into the domain of video image pre-processing. We aim to determine if the application of the Contrast Limited Adaptive Histogram Equalization (CLAHE) technique — a method adopted in a parallel study by **Chen et al.** — can refine the performance of hand-pose estimation algorithms. Chen et al. grappled with analogous challenges related to pose estimation errors in their research, `Patient-Specific Pose Estimation in Clinical Environments`, which revolved around bedridden patients. The belief is that a better pose estimation data, driven by nuanced pre-processing, might elevate the classifier's accuracy for hand tremor severity.

The foundational data for our exploration is the PDMotorDB dataset, a collection of videos portraying Parkinson's disease patients engaged in finger tapping tasks. Shared by Yang et al. in their 2022 paper "Automatic Detection Pipeline for Accessing the Motor Severity of Parkinson’s Disease in Finger Tapping and Postural Stability" this dataset, combined with insights from both Yang et al. and Chen et al., positions us to extract meaningful conclusions about the potential synergies between cutting-edge image processing techniques and AI-infused diagnostic tools.

PDMotorDB dataset <br>https://github.com/pddata/PDMotorDB

**Yang et al. (2022)** `Automatic Detection Pipeline for Accessing the Motor Severity of Parkinson’s Disease in Finger Tapping and Postural Stability` <br>https://doi.org/10.1109/ACCESS.2022.3183232

**Chen et al. (2018)** `Patient-Specific Pose Estimation in Clinical Environments` <br>https://doi.org/10.1109/JTEHM.2018.2875464


# Exploratory Data Analysis

In this part, we will replicate the methodology applied by Yang et al. to create all the domain knowledge features they used and train a DNN model to classify the severity of tremor from the hand videos. Hand landmarks were extracted using the MediaPipe library for both the right and left hand and save in a csv file. To visually explore the data, we first filter for only one participant at a time. Then, we create the features for all the videos of that participant and train a DNN model with the same architecture as the one used by Yang et al. to classify the severity of tremor. 

**From the paper (page 5)**

```
In the domain knowledge extraction stage, we extract three features: the tapping rate, the tapping frozen times, and the tapping amplitude variation for the finger tapping action. Those features represent the patient’s ability to control their fingers. 

The tapping rate is defined as the tapping number divided by the time interval. 
Tapping frozen times is defined as the hesitation number of the patient. 
The tapping amplitude variation refers to the distance variation in which the patient spread the thumb and the index finger. 

To calculate the three features above, we first calculate the distance from the thumb to the index finger for each frame. Finger tapping status can be represented by the distances. Those distances compose of a discrete signal, which can form a wave form by connecting the adjacent point. To make the wave form smooth and filter noise, we use low pass filtering to get rid of the frequency with over 4 Hz. Second, we normalize the distance by dividing by the maximum distance to avoid the variation of the hand distance to the camera. For example, when the distance comes to zero, the thumb and the index finger are pressed together. When the distance comes to one, the thumb and the index finger are pulled apart at most. The wave form of one cycle can be viewed as a tapping, shown in Figure 3(a). The mark ‘x’ in yellow represents the local minimal point.

From the wave form, the tapping rate can be calculated as the tapping number divided by time. For example, in the first row of Figure 3(b), the tapping rate is the number of ‘x’ mark, the local minimal point divided by five seconds. The tapping rate is 1.6 taps per second in this example. The frozen state is defined as the patient prolonging the current action for a while. In the second row of Figure 3(b), we can see there is an obvious flat line in the center of the wave form. We first calculate the first derivative of the wave form and take the range with small derivatives as the frozen interval. We take the number of the frozen interval as the feature of tapping frozen times. The tapping amplitude refers to the patient’s ability to keep the tapping amplitude. For example, there is an obvious decrease in the amplitude of the wave form, shown in the third row of Figure 3(b). We calculate the standard deviation of the tapping amplitude to grasp the tapping amplitude variation, avoiding the effects of the various absolute pixel distances. Besides, this is also consistent with the criteria in the 3.12 part of MDS-UPDRS.
```

### Reading the hand landmarks file

In [1]:
import pandas as pd

input_path = r'C:\Users\Eduardo\OneDrive\Área de Trabalho\marked-finger-lr'

df_landmark = pd.read_csv(f'{input_path}/df_landmark_confidence_0.5.csv', sep=';')
df_landmark.shape

(1591655, 12)

In [ ]:
df_landmark.head()

### Selecting one participant

After selecting a participant, we also filter the data to only include the tip of the index finger and the thumb, that we will use to create the features.

In [3]:
participant = 'R00001.avi'


# from the pose estimation algorithm, we get the following landmarks:
# landmark_id 4: THUMB_TIP
# landmark_id 8: INDEX_FINGER_TIP
# https://developers.google.com/mediapipe/solutions/vision/hand_landmarker

landmark_thumb_tip = 4
landmark_index_finger_tip = 8

data = df_landmark[(df_landmark['participant']==participant) & 
                   (df_landmark['landmark_id'].isin([landmark_thumb_tip, landmark_index_finger_tip]))].copy()

data['landmark_id'] = data['landmark_id'].map({landmark_thumb_tip: '_thumb_tip', landmark_index_finger_tip: '_index_tip'})

data.shape

(250, 12)

In [4]:
data.head()

,file,participant,frame,hand_num,landmark_id,x,y,z,result,result_index,result_score,result_label
955064,E:\PDMotorDB\lr\R00001.avi,R00001.avi,0,0.0,_thumb_tip,0.644310,0.340900,-0.096499,[classification {\n index: 0\n score: 0.9879...,0.0,0.987923,Left
955068,E:\PDMotorDB\lr\R00001.avi,R00001.avi,0,0.0,_index_tip,0.528733,0.202041,-0.137749,[classification {\n index: 0\n score: 0.9879...,0.0,0.987923,Left
955085,E:\PDMotorDB\lr\R00001.avi,R00001.avi,1,0.0,_thumb_tip,0.688530,0.376662,-0.187936,[classification {\n index: 0\n score: 0.9812...,0.0,0.981231,Left
955089,E:\PDMotorDB\lr\R00001.avi,R00001.avi,1,0.0,_index_tip,0.556952,0.117352,-0.204186,[classification {\n index: 0\n score: 0.9812...,0.0,0.981231,Left
955106,E:\PDMotorDB\lr\R00001.avi,R00001.avi,2,0.0,_thumb_tip,0.696156,0.393507,-0.192796,[classification {\n index: 0\n score: 0.9861...,0.0,0.986167,Left


### Calculating the distance between the thumb and the index finger

`To calculate the three features above, we first calculate the distance from the thumb to the index finger for each frame`

In [5]:
import numpy as np

data_pivot = data.pivot_table(index=['frame', 'participant'], columns='landmark_id', values=['x', 'y']).reset_index()
data_pivot.columns = [str(col[0]) + str(col[1]).replace('.0', '') for col in data_pivot.columns]

data_pivot['dist'] = np.sqrt((data_pivot['x_thumb_tip'] - data_pivot['x_index_tip'])**2 + 
                             (data_pivot['y_thumb_tip'] - data_pivot['y_index_tip'])**2)

data_pivot.shape

(125, 7)

In [6]:
data_pivot.head()

,frame,participant,x_index_tip,x_thumb_tip,y_index_tip,y_thumb_tip,dist
0,0,R00001.avi,0.528733,0.644310,0.202041,0.340900,0.180665
1,1,R00001.avi,0.556952,0.688530,0.117352,0.376662,0.290782
2,2,R00001.avi,0.566714,0.696156,0.090360,0.393507,0.329627
3,3,R00001.avi,0.581263,0.678055,0.095330,0.403044,0.322578
4,4,R00001.avi,0.509086,0.586647,0.167417,0.356662,0.204522


### Low pass filtering

`To make the wave form smooth and filter noise, we use low pass filtering to get rid of the frequency with over 4 Hz`

In [7]:
import numpy as np
from scipy.signal import butter, filtfilt

cutoff = 4 # cutoff frequency in Hz
sampling_rate = 25 # video in 25 fps
order = 2

# butterworth filter
normal_cutoff = cutoff / (0.5 * sampling_rate)
b, a = butter(order, normal_cutoff, btype='low', analog=False)

# applying filter
data_pivot['dist_filter'] = filtfilt(b, a, data_pivot['dist'])


### Normalizing the distance

`we normalize the distance by dividing by the maximum distance to avoid the variation of the hand distance to the camera`

In [8]:
from sklearn.preprocessing import minmax_scale
#data_pivot['dist_norm'] = minmax_scale(data_pivot['dist'])
data_pivot['dist_norm'] = minmax_scale(data_pivot['dist_filter'])

### Calculating local minima

`when the distance comes to zero, the thumb and the index finger are pressed together. When the distance comes to one, the thumb and the index finger are pulled apart at most. The wave form of one cycle can be viewed as a tapping, shown in Figure 3(a). The mark ‘x’ in yellow represents the local minimal point`

In [9]:
# https://stackoverflow.com/a/74964697/11233264
data_pivot['flag_minima_rolling'] = np.where(data_pivot['dist_norm'] == data_pivot['dist_norm'].rolling(5, center=True).min(), True, False)

data_pivot['flag_minima_rolling'].value_counts(dropna=False, normalize=False).head()

False    112
True      13
Name: flag_minima_rolling, dtype: int64

In [10]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_pivot['frame'], y=data_pivot['dist_norm'], mode='markers+lines', 
                         marker_color=data_pivot['flag_minima_rolling'].map({True: 'red', False: 'blue'})))

fig.update_layout(title=f'{participant.replace(".avi", "")} - Distance <b>WITH</b> filter').show()

In [11]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_pivot['frame'], y=data_pivot['dist'], mode='markers+lines', 
                         marker_color=data_pivot['flag_minima_rolling'].map({True: 'red', False: 'blue'})))

fig.update_layout(title=f'{participant.replace(".avi", "")} - Distance <b>WITHOUT</b> filter').show()

### Tapping rate

`the tapping rate can be calculated as the tapping number divided by time. For example, in the first row of Figure 3(b), the tapping rate is the number of ‘x’ mark, the local minimal point divided by five seconds. The tapping rate is 1.6 taps per second in this example.`

In [12]:
video_length_seconds = 5

tapping_rate = data_pivot['flag_minima_rolling'].sum() / video_length_seconds
tapping_rate

2.6

### Tapping frozen times

`The frozen state is defined as the patient prolonging the current action for a while. In the second row of Figure 3(b), we can see there is an obvious flat line in the center of the wave form. We first calculate the first derivative of the wave form and take the range with small derivatives as the frozen interval. We take the number of the frozen interval as the feature of tapping frozen times.`

In the paper the authors do not specify the threshold for the derivative. If the authors reply to my email and a different value is specified, this part will be updated.

In [13]:
frozen_threshold = 0.01



data_pivot['dist_derivative'] = data_pivot['dist_norm'].diff()

data_pivot['frozen_state'] = np.abs(data_pivot['dist_derivative']) < frozen_threshold

In [14]:
data_pivot['frozen_state'].value_counts()

False    122
True       3
Name: frozen_state, dtype: int64

In [15]:
data_pivot.head()

,frame,participant,x_index_tip,x_thumb_tip,y_index_tip,y_thumb_tip,dist,dist_filter,dist_norm,flag_minima_rolling,dist_derivative,frozen_state
0,0,R00001.avi,0.528733,0.644310,0.202041,0.340900,0.180665,0.180662,0.552908,False,NaN,False
1,1,R00001.avi,0.556952,0.688530,0.117352,0.376662,0.290782,0.268091,0.807485,False,0.254578,False
2,2,R00001.avi,0.566714,0.696156,0.090360,0.393507,0.329627,0.308954,0.926472,False,0.118987,False
3,3,R00001.avi,0.581263,0.678055,0.095330,0.403044,0.322578,0.278429,0.837587,False,-0.088885,False
4,4,R00001.avi,0.509086,0.586647,0.167417,0.356662,0.204522,0.188872,0.576815,False,-0.260772,False


In [16]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=data_pivot['frame'], y=data_pivot['dist_derivative'], mode='markers+lines', 
                         marker_color=data_pivot['frozen_state'].map({True: 'red', False: 'blue'})))

fig.update_layout(title=f'{participant.replace(".avi", "")} - Derivative and frozen states').show()

### Tapping amplitude variation

`The tapping amplitude refers to the patient’s ability to keep the tapping amplitude. For example, there is an obvious decrease in the amplitude of the wave form, shown in the third row of Figure 3(b). We calculate the standard deviation of the tapping amplitude to grasp the tapping amplitude variation, avoiding the effects of the various absolute pixel distances.`

In [17]:
data_pivot['dist_std'] = data_pivot['dist_norm'].std()
data_pivot['dist_std'].unique()

array([0.31315559])

### Creating Dataset for Model

Now that we have explored the data, we can create a dataset for our model for all the videos.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import minmax_scale
from scipy.signal import butter, filtfilt



input_path = r'C:\Users\Eduardo\OneDrive\Área de Trabalho\marked-finger-lr'
df_landmark = pd.read_csv(f'{input_path}/df_landmark_confidence_0.5.csv', sep=';')



landmark_thumb_tip = 4
landmark_index_finger_tip = 8
video_length_seconds = 5
dataset = []

for participant in df_landmark['participant'].unique():
    data = df_landmark[(df_landmark['participant']==participant) & 
                       (df_landmark['landmark_id'].isin([landmark_thumb_tip, landmark_index_finger_tip]))].copy()

    data['landmark_id'] = data['landmark_id'].map({landmark_thumb_tip: '_thumb_tip', landmark_index_finger_tip: '_index_tip'})

    if data.shape[0] == 250: # video hase 125 frames * 2 for index and thumb tip
        #print(participant)

        data_pivot = data.pivot_table(index=['frame', 'participant'], columns='landmark_id', values=['x', 'y']).reset_index()
        data_pivot.columns = [str(col[0]) + str(col[1]).replace('.0', '') for col in data_pivot.columns]

        # distance
        data_pivot['dist'] = np.sqrt((data_pivot['x_thumb_tip'] - data_pivot['x_index_tip'])**2 + 
                                    (data_pivot['y_thumb_tip'] - data_pivot['y_index_tip'])**2)
        
        # low pass filter
        cutoff = 4
        sampling_rate = 25
        order = 2
        normal_cutoff = cutoff / (0.5 * sampling_rate)
        b, a = butter(order, normal_cutoff, btype='low', analog=False)
        data_pivot['dist_filter'] = filtfilt(b, a, data_pivot['dist'])

        data_pivot['dist_norm'] = minmax_scale(data_pivot['dist_filter'])

        # local minima
        data_pivot['flag_minima_rolling'] = np.where(data_pivot['dist_norm'] == data_pivot['dist_norm'].rolling(5, center=True).min(), True, False)
        
        # tapping rate
        tapping_rate = data_pivot['flag_minima_rolling'].sum() / video_length_seconds
        tapping_rate

        # tapping frozen
        data_pivot['dist_derivative'] = data_pivot['dist_norm'].diff()

        frozen_threshold = 0.01

        data_pivot['frozen_state'] = np.abs(data_pivot['dist_derivative']) < frozen_threshold
        tapping_frozen = data_pivot[data_pivot['frozen_state']==True].shape[0]

        # tapping standard deviation
        tapping_std = data_pivot['dist_norm'].std()

        dataset.append({'participant': participant, 'tapping_rate': tapping_rate, 'tapping_std': tapping_std, 'tapping_frozen': tapping_frozen})

df_dataset = pd.DataFrame.from_records(dataset)
df_dataset['participant'] = df_dataset['participant'].str.replace('.avi', '', regex=False)
df_dataset.shape


In [ ]:
df_dataset.head()

### Target variable

Reading the target values from the separate text files.

In [ ]:
df_target = pd.DataFrame()

files = ["E:\PDMotorDB\lefthand_train.txt", "E:\PDMotorDB\lefthand_val.txt", r"E:\PDMotorDB\righthand_train.txt", r"E:\PDMotorDB\righthand_val.txt"]

for file in files:
    participant_prefix = 'L' if 'lefthand' in file else 'R'

    with open(file, 'r') as f:
        lines = f.readlines()
        df_lines = pd.DataFrame(lines, columns=['participant'])
        
        df_lines['target'] = df_lines['participant'].str.replace('\n', '').str.split(' ').str[1].astype(int)
        df_lines['participant'] = participant_prefix + df_lines['participant'].str.split(' ').str[0]
        df_lines['train_val'] = 'train' if 'train' in file else 'val'

        df_target = df_target.append(df_lines)
df_target.shape

In [ ]:
df_target.head()

In [ ]:
df_dataset = df_dataset.merge(df_target, on='participant', how='left')
df_dataset.shape

In [ ]:
df_dataset.head()

In [ ]:
# Removing dirty data from the dataset

exclude_ids = [ 'L00008', 'L00013', 'L00031', 'L00035', 'L00036', 'L00042', 'L00075', 'L00095', 'L00098', 'L00104', 'L00105', 'L00116', 'L00139', 'L00170', 'L00188', 'L00189', 'L00191', 'L00193', 'L00198', 'L00214', 'L00217', 'L00228', 'L00231', 'L00233', 'L00236', 'L00244', 'L00251', 'L00253', 'L00255', 'L00258', 'L00262', 'L00263', 'L00271', 'L00282', 'L00284', 'L00294', 'L00305', 'L00307', 'L00312', 'L00338', 'L00343', 'L00368', 'R00017', 'R00034', 'R00035', 'R00041', 'R00054', 'R00060', 'R00063', 'R00071', 'R00074', 'R00085', 'R00088', 'R00105', 'R00118', 'R00138', 'R00139', 'R00146', 'R00159', 'R00178', 'R00194', 'R00195', 'R00197', 'R00203', 'R00212', 'R00214', 'R00243',]

df_dataset = df_dataset[~df_dataset['participant'].isin(exclude_ids)]

### Model

**From the paper (page 8)**

`We use the data with three dimensions to train a classifier with a deep neural network (DNN). Our DNN classifier has `three layers and each layer has 8 hidden nodes`. We use `adam learner `with` learning rate: 0.01 and weight decay: 1e-6`. The number of the `training epoch is 200 `and the` training loss is Softmax`. Then we test the method on the test set. We report the `precision, recall, f1-score, and confusion matrix` of the DNN-based classifier. Type 0, type 1, type 2, and type 3 represent the normal, slight, mild, and moderate severity of MDS-UPDRS respectively.`

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, regularizers

def create_model():
    model = tf.keras.Sequential([
        layers.Input(shape=(3,)),
        layers.Dense(8, activation='softmax', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(8, activation='softmax', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(8, activation='softmax', kernel_regularizer=regularizers.l2(1e-6)),
        layers.Dense(4, activation='softmax')
    ])

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
                loss='categorical_crossentropy', 
                metrics=['accuracy',
                         tf.keras.metrics.Precision(),
                         tf.keras.metrics.Recall()])
    
    return model

In [ ]:
model = create_model()

### Train, test and validation split

Since the authors separate the data between left and right hand, we will do the same here with the _selected\_hand_ variable. We will also split the data into train, test and validation sets using the same proportions as the authors.

In [ ]:
selected_hand = 'R' # 'L'

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

train_dataset = df_dataset[(df_dataset['train_val']=='train') & (df_dataset['participant'].str.contains(selected_hand))]


test_dataset = df_dataset[(df_dataset['train_val']=='val') & (df_dataset['participant'].str.contains(selected_hand))]


X_train, X_val, y_train, y_val = train_test_split(train_dataset[['tapping_rate', 'tapping_std', 'tapping_frozen']], train_dataset['target'], test_size=0.4, random_state=42)

print(f'''
Train shape: {X_train.shape} {y_train.shape} \t {np.unique(y_train, return_counts=True)}
Val shape:  {X_val.shape} {y_val.shape} \t {np.unique(y_val, return_counts=True)}
''')

y_train = to_categorical(y_train, num_classes=df_dataset['target'].nunique())
y_val = to_categorical(y_val, num_classes=df_dataset['target'].nunique())


In [ ]:
X_train.head()

In [ ]:
y_train[:5]

Training the model for 200 epochs

In [ ]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200)

In [ ]:
import plotly.graph_objects as go

trace_train = go.Scatter(
    x=[i for i in range(len(history.history['loss']))],
    y=history.history['loss'],
    mode='lines', name='Train Loss'
)

trace_test = go.Scatter(
    x=[i for i in range(len(history.history['val_loss']))],
    y=history.history['val_loss'],
    mode='lines', name='Test Loss'
)

fig = go.Figure(data=[trace_train, trace_test]).update_layout(title=f'Train and Validation Loss over Epochs for <b>{selected_hand} hand</b>', xaxis_title='Epoch', yaxis_title='Loss')
fig.show()

# Running multiple times

In order to get a better understanding of the model results, we will run the model multiple times and calculate the average of the metrics used by the authors.

In [ ]:
from sklearn.metrics import classification_report

import random
import warnings
warnings.filterwarnings('ignore')

num_runs = 100
list_results = []

for run in range(num_runs):
    print(f'Run: {run}')
    X_train, X_val, y_train, y_val = train_test_split(train_dataset[['tapping_rate', 'tapping_std', 'tapping_frozen']], train_dataset['target'], test_size=0.4, random_state=random.randint(0, 100))#42)

    y_train = to_categorical(y_train, num_classes=df_dataset['target'].nunique())
    y_val = to_categorical(y_val, num_classes=df_dataset['target'].nunique())

    #model.fit(X_train, y_train, epochs=200, verbose=0)
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=200, verbose=0)

    X_test = test_dataset[df_dataset['participant'].str.contains(selected_hand)][['tapping_rate', 'tapping_std', 'tapping_frozen']]
    y_test = to_categorical(test_dataset[df_dataset['participant'].str.contains(selected_hand)]['target'], num_classes=df_dataset['target'].nunique())
    
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    cr = classification_report(y_true, y_pred_classes, zero_division=0, output_dict=True)
    print(cr)

    run_precision =  cr['macro avg']['precision']
    run_recall = cr['macro avg']['recall']
    run_f1 = cr['macro avg']['f1-score']
    list_results.append({'run': run, 'precision': run_precision, 'recall': run_recall, 'f1': run_f1, 
                         'precision_0': cr['0']['precision'], 'recall_0': cr['0']['recall'], 'f1_0': cr['0']['f1-score'], 
                         'precision_1': cr['1']['precision'], 'recall_1': cr['1']['recall'], 'f1_1': cr['1']['f1-score'], 
                         'precision_2': cr['2']['precision'], 'recall_2': cr['2']['recall'], 'f1_2': cr['2']['f1-score'], 
                         'precision_3': cr['3']['precision'], 'recall_3': cr['3']['recall'], 'f1_3': cr['3']['f1-score']})
    print()

In [ ]:
df_runs = pd.DataFrame.from_records(list_results)
df_runs.shape

In [ ]:
df_runs.head()

In [ ]:
df_runs[['precision', 'recall', 'f1']].describe()

In [ ]:
selected_hand

In [ ]:
df_runs.describe()

model = '' #'model_20230808141544'

df_runs.to_csv(f'runs_{model}.csv', index=False)
df_runs.describe().to_csv(f'runs_describe_{model}.csv', index=False)

In [ ]:
X_test = test_dataset[df_dataset['participant'].str.contains(selected_hand)][['tapping_rate', 'tapping_std', 'tapping_frozen']]
y_test = to_categorical(test_dataset[df_dataset['participant'].str.contains(selected_hand)]['target'], num_classes=df_dataset['target'].nunique())

y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

from sklearn.metrics import classification_report, confusion_matrix

cr = classification_report(y_true, y_pred_classes, zero_division=0)
print(cr)

cm = confusion_matrix(y_true, y_pred_classes)
print(cm)
print()

cm_pct = (cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]).round(4)
print(cm_pct)

In [ ]:
import datetime

model.save(f'model_{datetime.datetime.now().strftime("%Y%m%d%H%M%S")}.h5')

In [ ]:
model.summary()

In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model.png', expand_nested=True, show_shapes=True, show_layer_activations=True, show_dtype=True, show_layer_names=True, rankdir='TB', dpi=96, )